# The Array-Backed List

## Agenda

1. The List **Abstract Data Type** (ADT)
2. A List **Data Structure**
3. The List API
4. Getting started
5. NumPy arrays
6. The `ArrayList` data structure
7. Runtime analysis

## 1. The List **Abstract Data Type** (ADT)

An **abstract data type (ADT)** defines a *conceptual model* for how data may be stored and accessed.

A **list ADT** is a data container where:

- values are ordered in a *sequence*
- each value has at most one preceding and one succeeding value
- a given value may appear more than once in a list

Other common ADTs (some of which we'll explore later) include:

- Stacks
- Queues
- Priority Queues
- Maps
- Graphs

## 2. A List **Data Structure**

A **list data structure** is a *concrete implementation* of the list ADT in some programming language, which, in addition to adhering to the basic premises of the ADT, will also typically support operations that:

- access values in the list by their position (index)
- append and insert new values into the list
- remove values from the list

The implementation of any data structure will generally rely on simpler, constituent data types (e.g., "primitive" types offered by the language), the choice of which may affect the runtime complexities of said operations.

## 3. The List API

The operations we'll be building into our list data structures will be based on the [common](https://docs.python.org/3.6/library/stdtypes.html#common-sequence-operations) and [mutable](https://docs.python.org/3.6/library/stdtypes.html#mutable-sequence-types) sequence operations defined by the Python library.

In [ ]:
class List:        
    ### subscript-based access ###
    
    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        pass

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        pass

    def __delitem__(self, idx):
        """Implements `del self[idx]`"""
        pass
    
    ### stringification ###
            
    def __repr__(self):
        """Supports inspection"""
        return '[]'
    
    def __str__(self):
        """Implements `str(self)`"""
        return '[]'

    ### single-element manipulation ###
    
    def append(self, value):
        pass
    
    def insert(self, idx, value):
        pass
    
    def pop(self, idx=-1):
        pass
    
    def remove(self, value):
        pass
    
    ### predicates (T/F queries) ###
    
    def __eq__(self, other):
        """Implements `self == other`"""
        return True

    def __contains__(self, value):
        """Implements `val in self`"""
        return True
    
    ### queries ###
    
    def __len__(self):
        """Implements `len(self)`"""
        return len(self.data)
    
    def min(self):
        pass
    
    def max(self):
        pass
    
    def index(self, value, i, j):
        pass
    
    def count(self, value):
        pass

    ### bulk operations ###

    def __add__(self, other):
        """Implements `self + other_array_list`"""
        return self
    
    def clear(self):
        pass
    
    def copy(self):
        pass

    def extend(self, other):
        pass

    ### iteration ###
    
    def __iter__(self):
        """Supports iteration (via `iter(self)`)"""
        pass

## 4. Getting started

In [ ]:
class List:
    def append(self, value):
        pass
    
    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        pass

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        pass
    
    def __repr__(self):
        """Supports inspection"""
        pass

In [ ]:
l = List()
l.append(42)

In [ ]:
l # equivalent to l.__repr__()

In [ ]:
l[0] # equivalent to l.__getitem__(0)

In [ ]:
l[0] = 331 # equivalent to l.__setitem__(0, 331)

In [ ]:
l[500]

In [ ]:
l

Of course, we need our list implementation to hold more than a single element. The obvious solution for this is to use an *array*.

## 5. NumPy arrays

Python does not come with a built-in array type. Instead, we're going to make use of the array implementation provided by the [NumPy scientific computing package](https://numpy.org/doc/stable/user/absolute_beginners.html).

To create a NumPy array of size N, we can do:

In [ ]:
import numpy as np

N = 10
arr = np.empty(N, dtype=object)

The `dtype=object` specification indicates that we want to use the array to store references to arbitrary Python objects. The `empty` function creates an array of the specified size, but leaves all elements uninitialized.

In [ ]:
for i in range(5):
    arr[i] = i

arr[0]  = 'hello'
arr[4] = 'world'

In [ ]:
arr

In [ ]:
len(arr)

Recall that arrays are **fixed-size**, so we cannot append, insert, or delete elements to/from them directly. These operations must be implemented by the data structure we build *on top of* the array.

I.e., *an array is not a data structure*! It is a low-level *data storage* mechanism.

## 6. The `ArrayList` data structure

Our list data structure will make use of a NumPy array as its backing data store. This array will have a fixed initial size, but as elements are added to the list it may become necessary to create a new, larger backing array and copy our elements over. Each time we create a new backing array, we will *double the capacity of the original* -- this may seem arbitrary now, but we will see why it is a critical detail later on!

In [ ]:
import numpy as np

class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0 # number of elements in the list


    def append(self, value):
        pass


    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        assert isinstance(idx, int), 'Index must be an integer'
        pass
    

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        assert isinstance(idx, int), 'Index must be an integer'
        pass
        

    def __delitem__(self, idx):
        """Implements `del self[idx]`"""
        assert isinstance(idx, int), 'Index must be an integer'
        pass
        
    
    def __len__(self):
        """Implements `len(self)`"""
        return self.size # note: not the same as `len(self.data)`!
    
    
    def __repr__(self):
        """Supports inspection"""
        repr_elems = [repr(self.data[i]) for i in range(self.size)]
        return '[' + ', '.join(repr_elems) + ']'

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(x)
l

In [ ]:
l[0] = 'hello'
l[-1] = 'world'
l

In [ ]:
l[2] = 'two'
l[-3] = 'seven'
l

In [ ]:
del l[5]
l

In [ ]:
del l[0]
del l[-1]
l

In [ ]:
len(l)

## 7. Runtime analysis

- Indexing: $O(?)$

- Search (unsorted): $O(?)$

- Search (sorted): $O(?)$

- Deletion: $O(?)$

- Append: $O(?)$
         
- Insertion: $O(?)$

### Not doubling?

If we chose to expand the array by a constant amount instead of doubling it, what would be the amortized runtime for append?

In [ ]:
import numpy as np

class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0


    def append(self, value, doubling=True):
        if self.size == len(self.data):
            if doubling:
                nsize = 2 * len(self.data)
            else:
                nsize = len(self.data) + 1000

            ndata = np.empty(nsize, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata
            
        self.data[self.size] = value
        self.size += 1

In [ ]:
# plot average runtime of `append` for doubling vs. non-doubling strategies,
# as a function of the total number of elements appended

import timeit
import matplotlib.pyplot as plt

ns = np.linspace(1000, 10_000, 50, dtype=int)
ts1 = [timeit.timeit(stmt=f'for _ in range({n}):'
                          f'    lst.append(None, doubling=True)', 
                     setup='lst = ArrayList()',
                     globals=globals(), 
                     number=10) / n
       for n in ns]
ts2 = [timeit.timeit(stmt=f'for _ in range({n}): '
                          f'    lst.append(None, doubling=False)', 
                     setup='lst = ArrayList()',
                     globals=globals(), 
                     number=10) / n
       for n in ns]

plt.plot(ns, ts1, 'ob')
plt.plot(ns, ts2, 'or');

Note how doubling the array size (in blue) gives us a *constant amortized runtime*.

Not doubling the array size (in red) gives us a *linear amortized runtime*! (We won't prove this directly, but can easily observe it.)